# __Fake News Prediction__
### __Uses:__ Logistic Regression Model

*__Made by:__ Sebastián Cervantes ^^*

## **Work flow:**
 * News Data (Kaggle)
 * Data pre processing
 * Training test split
 * Logistic Regression algorythm
 * Evaluation
 
 ## __Consideretions__
 1. Id: unique ID for news
 2. Title: title of the article
 3. author: who wrote the article
 4. Text: the content of the new/article
 5. label: marks wheater the news articles is real or fake
 
 1: Fake new
 0: Real new

## __Importing the dependencies__

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #text into feature vector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cebastian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### __Data pre-processing__

In [4]:
# loading the news into pandas dataset
news_dataset = pd.read_csv('train.csv')

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# Missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
#Replacing missing values with empy string. Other solution could be delete them. But is more apropiate the first option
news_dataset = news_dataset.fillna('')

In [9]:
# Mergin title and author name into just one column. It's easier to process this two fields for the model
# Another option is analyse onle the content of the article but this will demand more resources of the machine
# And analyzing the 3 fields is the most optimal choise but will demand a lot of time and resources
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [10]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


### __Training test split__

In [11]:
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

__Stemming__
Process fo reducing a word to its root word. i.e: actor, actress, actor --> act

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # substitute: numbers, puntuations with empty strings
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split() # convert the text into list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #deleting stop-words
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [16]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [17]:
# Separating data from label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [18]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [19]:
print(Y)

[1 0 1 ... 0 1 1]


In [20]:
Y.shape

(20800,)

### How a logistic regression needs numerical data, I use TfidVectorizer to transform text to numerical data

In [21]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X= vectorizer.transform(X)

In [22]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [23]:
X_train, X_test, Y_train, Y_test, = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

### __Logistic regression Algorythm__

In [24]:
model= LogisticRegression()

In [25]:
model.fit(X_train, Y_train)

LogisticRegression()

### __Evaluation__

In [26]:
# training accuracy score
X_train_prediction = model.predict(X_train)
train_score = accuracy_score(X_train_prediction, Y_train)

In [27]:
print('Accuraccy score of the training data: ',train_score)

Accuraccy score of the training data:  0.9865985576923076


In [28]:
## testing accuraccy score
X_test_prediction = model.predict(X_test)
test_score = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the training data: ',test_score)

Accuracy score of the training data:  0.9790865384615385


In [29]:
print(X_test)

  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485
  (1, 16996)	0.09117761343372983
  (1, 15295)	0.08946281236254729
  (1, 14046)	0.42524648908354634
  (1, 13190)	0.36773046084789346
  (1, 12741)	0.24868518461414146
  (1, 12279)	0.3796661151115819
  (1, 12041)	0.37327055071909065
  (1, 10306)	0.08813410128297053
  (1, 8813)	0.42524648908354634
  (1, 4008)	0.23098933893199997
  (1, 3339)	0.2834482751186189
  (2, 16868)	0.344315415802567
  :	:
  (4158, 16782)	0.2796201559111399
  (4158, 13954)	0.2713157796360236
  (4158, 13918)	0.29432702492466434
  (4158, 13775)	0.21479309786895925
  (4158, 13600)	0.2496550139728973
  (4158, 11068)	0.2943

### __Aplication__

In [30]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [31]:
print(X_test[3])

  (0, 14632)	0.23337028911399943
  (0, 12693)	0.211577404470594
  (0, 11888)	0.25336998543209355
  (0, 11246)	0.340188979812318
  (0, 11217)	0.3302474074203597
  (0, 7668)	0.18748490820807126
  (0, 6868)	0.2729431954288567
  (0, 5529)	0.2895160407717208
  (0, 3782)	0.5146474416282095
  (0, 2806)	0.22586908309808348
  (0, 2510)	0.19575583865343715
  (0, 2407)	0.22941655373198824
  (0, 1894)	0.10717190161988106


### Using test csv

In [32]:
testing_dataset = pd.read_csv('test.csv')
testing_dataset.shape

(5200, 4)

In [33]:
testing_dataset.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [34]:
testing_dataset=testing_dataset.fillna('')
testing_dataset.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [35]:
testing_dataset['content'] = testing_dataset['author']+' '+testing_dataset['title']
print(testing_dataset['content'])

0       David Streitfeld Specter of Trump Loosens Tong...
1        Russian warships ready to strike terrorists n...
2       Common Dreams #NoDAPL: Native American Leaders...
3       Daniel Victor Tim Tebow Will Attempt Another C...
4       Truth Broadcast Network Keiser Report: Meme Wa...
                              ...                        
5195    Jody Rosen The Bangladeshi Traffic Jam That Ne...
5196    Sheryl Gay Stolberg John Kasich Signs One Abor...
5197    Mike McPhate California Today: What, Exactly, ...
5198     300 US Marines To Be Deployed To Russian Bord...
5199    Teddy Wayne Awkward Sex, Onscreen and Off - Th...
Name: content, Length: 5200, dtype: object


In [36]:
testing_dataset['content'] = testing_dataset['content'].apply(stemming)

In [37]:
test_two = testing_dataset['content'].values

In [38]:
print(test_two)

['david streitfeld specter trump loosen tongu purs string silicon valley new york time'
 'russian warship readi strike terrorist near aleppo'
 'common dream nodapl nativ american leader vow stay winter file lawsuit polic'
 ... 'mike mcphate california today exactli sushi new york time'
 'us marin deploy russian border norway'
 'teddi wayn awkward sex onscreen new york time']


In [39]:
vectorizer_new = TfidfVectorizer()
vectorizer_new.fit(test_two)
test_two= vectorizer.transform(test_two)

In [40]:
print(test_two)

  (0, 16996)	0.08864595251126647
  (0, 16088)	0.3205594987716609
  (0, 15582)	0.11032513315099028
  (0, 15368)	0.38547947461491155
  (0, 15295)	0.08697876504498342
  (0, 14605)	0.3970837204511862
  (0, 14591)	0.34467447913058513
  (0, 13850)	0.32956045112215354
  (0, 12030)	0.38547947461491155
  (0, 10306)	0.08568694729690247
  (0, 8842)	0.36912420870480717
  (0, 3623)	0.209228842739656
  (1, 16473)	0.4921981982038152
  (1, 15142)	0.3550835494981517
  (1, 14603)	0.35559908473568963
  (1, 13048)	0.3052969733971102
  (1, 12300)	0.3838422559012773
  (1, 10219)	0.37818137822242154
  (1, 347)	0.3487744590431239
  (2, 16746)	0.3016628915147435
  (2, 16354)	0.2909537422615255
  (2, 14457)	0.30637021801826614
  (2, 11598)	0.21447077225895875
  (2, 10419)	0.35982822390508096
  (2, 10176)	0.3194727535249622
  :	:
  (5196, 5884)	0.2500401085058591
  (5196, 1517)	0.19940396943572683
  (5196, 41)	0.2656503884474958
  (5197, 16996)	0.11797520134546996
  (5197, 15336)	0.33413616793311524
  (5197, 152

In [42]:
prediction = model.predict(test_two)

In [43]:
print(prediction)

[0 1 1 ... 0 1 0]
